In [ ]:
import re
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json

objects = []
articles = [
    ['cbt', "https://en.wikipedia.org/wiki/Cognitive_behavioral_therapy"],
    ['vr', "https://en.wikipedia.org/wiki/Virtual_reality"],
    ['ai', "https://en.wikipedia.org/wiki/Artificial_intelligence"]
]
for [topic, url] in articles:
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    paras = []
    for para in soup.find(id="mw-content-text").find_all('p'):
        para = para.get_text()
        para = para.replace('\n', ' ')
        para = re.sub(r"\[[0-9]+\]", "", para)
        if len(para) < 10:
            continue
        # para = re.sub(r"\[[0-9]*\]", "", para)
        if len(paras) == 0:
            paras.append(para)
        elif len(para) < 100:
            paras[len(paras) - 1] += para
        else:
            paras.append(para)
    for i, para in enumerate(paras):
        doc = dict(
            text=para,
            title=f"{topic}{i}"
        )
        objects.append(doc)

with open("mock_entries.json", "w") as f:
    f.write(json.dumps(objects))

In [ ]:
USE_GPU = False
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import json

from haystack.document_stores.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore.load('./index.faiss', './index.config')

In [ ]:
from haystack.nodes import DensePassageRetriever
dpr_retriever = DensePassageRetriever(
    document_store=document_store,
    use_gpu=False
    # query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    # passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)

In [ ]:
from haystack.nodes import RAGenerator
rag_generator = RAGenerator(
    model_name_or_path="facebook/rag-sequence-nq",
    retriever=dpr_retriever,
    top_k=3,
    min_length=3
)

In [ ]:
from haystack.pipelines import GenerativeQAPipeline

from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)
pipe = ExtractiveQAPipeline(reader, dpr_retriever)
result = pipe.run(query='What is CBT?', params={"Retriever": {"top_k": 5}})
a = 1

# # Change `minimum` to `medium` or `all` to raise the level of detail
# print_answers(result, details="all")
print(result)
return None

In [1]:
from txtai.embeddings import Embeddings

# Create embeddings model, backed by sentence-transformers & transformers
embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2", "content":True})

data = {
  "uid1": {"text": "US tops 5 million confirmed virus cases", "obj_id": "obj_id1"},
  "uid1": {"text": "Canada's last fully intact ice shelf has suddenly collapsed", "obj_id": "obj_id2"},
  "uid1": {"text": "forming a Manhattan-sized iceberg", "obj_id": "obj_id3"},
  "uid1": {"text": "Beijing mobilises invasion craft along coast as Taiwan tensions escalate", "obj_id": "obj_id4"},
  "uid1": {"text": "The National Park Service warns against sacrificing slower friends ", "obj_id": "obj_id5"},
  "uid1": {"text": "in a bear attack", "obj_id": "obj_id6"},
  "uid1": {"text": "Maine man wins $1M from $25 lottery ticket", "obj_id": "obj_id7"},
  "uid1": {"text": "Make huge profits without work, earn up to $100,000 a day", "obj_id": "obj_id8"}
}

embeddings.index([
    (uid, obj, None) 
    for uid, obj in data.items()
])

In [17]:
from txtai.embeddings.transform import Transform
import tempfile
from uuid import uuid4
documents = [
    (str(uuid4()), doc['text'], None) 
    for uid, doc in data.items()
]
transform = Transform(embeddings, 3)
with tempfile.NamedTemporaryFile(mode="wb", suffix=".npy") as buffer:
        # Load documents into database and transform to vectors
        ids, dimensions, vecs = transform(documents, buffer)
        if ids:
            # Normalize embeddings
            embeddings.normalize(vecs)

            # Save index dimensions
            embeddings.config["dimensions"] = dimensions
vecs

IntegrityError: UNIQUE constraint failed: sections.indexid

In [16]:
embeddings.search("select id, text, obj_id, score from txtai where similar('ice shelf')")

[]

In [19]:
embeddings.save('./tmp_stuff')

In [20]:
embeddings.embeddings

AttributeError: 'Embeddings' object has no attribute 'embeddings'